In [67]:
import tensorflow as tf
import pickle
from PIL import Image
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import cv2
from sklearn.utils.class_weight import compute_class_weight

In [2]:
#hyper parameters:
BATCH_SIZE = 32
NUM_EPOCHS = 20

### Model Architecture

In [58]:
class CardPredictor(tf.keras.Model):
    def __init__(self, num_of_classes):
        super(CardPredictor, self).__init__()
        
        data_augmentation = tf.keras.models.Sequential([
                                tf.keras.layers.RandomRotation(0.1),
                                tf.keras.layers.RandomZoom(0.1),
                            ])
        
        self.architecture = [        
                tf.keras.layers.InputLayer((300, 300, 3)),
                data_augmentation,

                tf.keras.layers.Conv2D(64, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(128, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(256, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(512, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),
                
                tf.keras.layers.Dropout(0.3),
                tf.keras.layers.Conv2D(1024, (3, 3)),       # Conv + ReLU
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPooling2D((2, 2)),

                tf.keras.layers.GlobalAveragePooling2D(),                                   # Flatten to vector
                
                tf.keras.layers.Dense(256, activation='relu'),               # Fully connected layer
                tf.keras.layers.Dropout(0.5),                                # Prevent overfitting
                tf.keras.layers.Dense(num_of_classes, activation='softmax')
                ]
        
        
        self.sequential = tf.keras.Sequential(self.architecture, name="card_predictor_classes_"+ str(num_of_classes))
        
    def call(self, x):
        """ Passes input image through the network. """
        return self.sequential(x)

# @tf.keras.saving.register_keras_serializable()
# class CardPredictor(tf.keras.Model):
#     def __init__(self, num_of_classes):
#         super(CardPredictor, self).__init__()
        
#         data_augmentation = tf.keras.models.Sequential([
#                                 tf.keras.layers.RandomRotation(0.1),
#                                 tf.keras.layers.RandomZoom(0.1),
#                             ])
        
#         self.architecture = [        
#                 tf.keras.layers.InputLayer((300, 300, 3)),
#                 data_augmentation,
                             
#                 tf.keras.layers.Conv2D(32, (3, 3)),       # Conv + ReLU
#                 tf.keras.layers.BatchNormalization(),
#                 tf.keras.layers.ReLU(),
#                 tf.keras.layers.MaxPooling2D((2, 2)),

#                 tf.keras.layers.Dropout(0.3),
#                 tf.keras.layers.Conv2D(64, (3, 3)),       # Conv + ReLU
#                 tf.keras.layers.BatchNormalization(),
#                 tf.keras.layers.ReLU(),
#                 tf.keras.layers.MaxPooling2D((2, 2)),

#                 tf.keras.layers.Dropout(0.3),
#                 tf.keras.layers.Conv2D(128, (3, 3)),       # Conv + ReLU
#                 tf.keras.layers.BatchNormalization(),
#                 tf.keras.layers.ReLU(),
#                 tf.keras.layers.MaxPooling2D((2, 2)),
                
#                 tf.keras.layers.Dropout(0.3),
#                 tf.keras.layers.Conv2D(256, (3, 3)),       # Conv + ReLU
#                 tf.keras.layers.BatchNormalization(),
#                 tf.keras.layers.ReLU(),
#                 tf.keras.layers.MaxPooling2D((2, 2)),
                
#                 tf.keras.layers.Dropout(0.3),
#                 tf.keras.layers.Conv2D(512, (3, 3)),       # Conv + ReLU
#                 tf.keras.layers.BatchNormalization(),
#                 tf.keras.layers.ReLU(),
#                 tf.keras.layers.MaxPooling2D((2, 2)),

#                 tf.keras.layers.GlobalAveragePooling2D(),                                   # Flatten to vector
                
#                 tf.keras.layers.Dense(256, activation='relu'),               # Fully connected layer
#                 tf.keras.layers.Dropout(0.5),                                # Prevent overfitting
#                 tf.keras.layers.Dense(num_of_classes, activation='softmax')
#                 ]
        
        
#         self.sequential = tf.keras.Sequential(self.architecture, name="card_predictor_classes_"+ str(num_of_classes))
        
#     def call(self, x):
#         """ Passes input image through the network. """
#         return self.sequential(x)

### Dataset Loading

In [12]:
#TRAIN
with open('train.pkl', 'rb') as file:
    train_data = pickle.load(file)
    
train_suit_data = {}
train_rank_data = {}

for key, inner_dict in train_data.items():
    img_path = inner_dict['img_path']
    value = inner_dict['concept_label']
    rank = value[:13]
    suit = value[13:]
    train_rank_data[img_path] = rank
    train_suit_data[img_path] = suit
    
with open('val.pkl', 'rb') as file:
    test_data = pickle.load(file)

#TEST
test_suit_data = {}
test_rank_data = {}

for key, inner_dict in test_data.items():
    img_path = inner_dict['img_path']
    value = inner_dict['concept_label']
    rank = value[:13]
    suit = value[13:]
    test_rank_data[img_path] = rank
    test_suit_data[img_path] = suit

In [39]:
# Define the image loading and preprocessing function
def load_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [300, 300])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Preprocess the dataset for training
train_rank_dataset = (
    tf.data.Dataset.from_tensor_slices((list(train_rank_data.keys()), list(train_rank_data.values())))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=10000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
train_suit_dataset = (
    tf.data.Dataset.from_tensor_slices((list(train_suit_data.keys()), list(train_suit_data.values())))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=10000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
test_rank_dataset = (
    tf.data.Dataset.from_tensor_slices((list(test_rank_data.keys()), list(test_rank_data.values())))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=10000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
test_suit_dataset = (
    tf.data.Dataset.from_tensor_slices((list(test_suit_data.keys()), list(test_suit_data.values())))
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=10000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

### Model Initialization

In [59]:
suit_model = CardPredictor(4)
suit_model.build((None, 300, 300, 3))
suit_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
rank_model = CardPredictor(13)
rank_model.build((None, 300, 300, 3))
rank_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

### Model Training

In [63]:
suit_model.fit(train_suit_dataset, 
                validation_data=test_suit_dataset, 
                epochs=NUM_EPOCHS, 
                verbose=1,
                callbacks=[
                    tf.keras.callbacks.ModelCheckpoint(
                        filepath='suit_model_ckpt.keras',
                        save_best_only=True,
                    )
                ])

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 95s 391ms/step - accuracy: 0.9580 - loss: 0.1467 - val_accuracy: 0.9867 - val_loss: 0.0653
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 98s 404ms/step - accuracy: 0.9747 - loss: 0.1042 - val_accuracy: 0.9937 - val_loss: 0.0652
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 99s 406ms/step - accuracy: 0.9843 - loss: 0.0702 - val_accuracy: 0.9970 - val_loss: 0.0265
Epoch 4/20
132/219 ━━━━━━━━━━━━━━━━━━━━ 30s 355ms/step - accuracy: 0.9959 - loss: 0.0163

KeyboardInterrupt: 

In [60]:
rank_model.fit(train_rank_dataset, 
                validation_data=test_rank_dataset, 
                epochs=NUM_EPOCHS, 
                verbose=1,
                callbacks=[
                    tf.keras.callbacks.ModelCheckpoint(
                        filepath='rank_model_ckpt.keras',
                        save_best_only=True,
                    )
                ])

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 96s 392ms/step - accuracy: 0.1187 - loss: 2.7555 - val_accuracy: 0.2444 - val_loss: 1.8915
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 98s 405ms/step - accuracy: 0.3767 - loss: 1.5873 - val_accuracy: 0.6751 - val_loss: 0.9267
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 95s 392ms/step - accuracy: 0.6294 - loss: 0.9892 - val_accuracy: 0.4980 - val_loss: 1.4789
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 95s 394ms/step - accuracy: 0.7144 - loss: 0.7542 - val_accuracy: 0.7636 - val_loss: 0.6938
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 96s 397ms/step - accuracy: 0.8193 - loss: 0.5371 - val_accuracy: 0.8906 - val_loss: 0.3997
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 96s 395ms/step - accuracy: 0.8391 - loss: 0.5339 - val_accuracy: 0.9019 - val_loss: 0.3805
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 97s 401ms/step - accuracy: 0.8367 - loss: 0.6210 - val_accuracy: 0.8743 - val_loss: 0.5733
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 95s 391ms/step - accuracy: 0.8665 - loss: 0

KeyboardInterrupt: 

### Model Load From CheckPoint

In [64]:
suit_model.load_weights("suit_model_ckpt.keras")

In [65]:
rank_model.load_weights("rank_model_ckpt.keras")